<a href="https://colab.research.google.com/github/BankNatchapol/ML-Algorithm/blob/master/MultiVar_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiple Variable Linear Regression**
---
Multiple variable linear regression คือ โมเดลทางสถิติที่สามารถบอกถึงความสัมพันธ์ ของตัวแปลต้น "หลายตัว" และตัวแปรตามได้ 
ซึ่งสามารถเขียน **hypothesis function** ได้ว่า <br>
\begin{equation}
h_\theta (x) = \theta _0 + \theta _1x_1 +\theta _2x_2 + \theta_3x_3+...+\theta_nx_n
\end{equation} <br>
ซึ่งถ้าเรามองว่า $\theta$ คือ vector <$\theta _0 , \theta _1,\theta _2 , \theta_3,...,\theta_n$><br>
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ x คือ vector <$x_0 , x_1,x_2 , x_3,...,x_n$><br><br>
จะได้ว่า
\begin{equation}
h_\theta (x) = \theta^Tx 
\end{equation} <br>
<br>
 สามารถเขียน **cost function** หรือ loss function ได้ว่า <br><br> 
\begin{equation}
J(\theta) = \frac{1}{2m}\sum^m_{i=1}(h_\theta(x^{(i)}) - y^{(i)})^2
\end{equation}<br><br>

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
pd.set_option('display.max_rows', df.shape[0]+1) 

โดยขั้นแรกจะทำการโหลด boston housing datasets ด้วย library sklearn 

In [0]:
dat = load_boston(return_X_y=False) # boston housing datasets

In [0]:
df = pd.DataFrame(dat.data,columns=dat.feature_names) # create dataframe
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


เป้าหมายของเราคือการทำนายราคาของบ้าน จากข้อมูล

In [0]:
target = pd.DataFrame(dat.target.T,columns=['PRICES']) # target prices
target.head()

,PRICES
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


โดยจากที่เห็นตัวข้อมูลจะพบว่า ข้อมูลในแต่ละ feature มี scale ที่ต่างกันมาก เลยมากทำการ scaling ด้วย **Z-score Normalization**<br><br>
\begin{equation}
x_j := \frac{x_j - \mu_j}{\sigma_j}
\end{equation} <br>
$\mu_j$ คือ ค่าเฉลี่ยของ feature นั้นๆ<br>
$\sigma_j$ คือ ส่วนเบี่ยงเบนมาตรฐานของ feature นั้นๆ

In [0]:
def scaling(x): #scaling input 
  std = np.std(x)
  std[0] = 1
  mean = np.mean(x)
  mean[0] = 0
  return (x-mean)/std
df = scaling(df)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,0.02731,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,0.02729,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,0.03237,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,0.06905,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


ต่อจากนั้นก็มาทำสมการ **hypothesis function**, **cost function** และ **gradient descent** <br><br>
\begin{equation}
Hypothesis\;Function :\;\;
h_\theta (x) = \theta^Tx 
\end{equation} <br>

\begin{equation}
Cost\;Function\;:\;\;
J(\theta) = \frac{1}{2m}\sum^m_{i=1}(h_\theta(x^{(i)}) - y^{(i)})^2
\end{equation}<br>
\begin{equation}
Gradient\;Descent\;:\;\;
\theta_j := \theta_j-\alpha\frac{1}{m}\sum^m_{i=1}(h_\theta(x^{(i)}) - y^{(i)})x^{(i)}_j
\end{equation}<br><br>

In [0]:
ones = pd.DataFrame({'ones':np.ones(df.shape[0]).T}) 
df = ones.join(df)
m = df.shape[0]
n = df.shape[1]
s = np.zeros(n)
sT = np.array([s]).T #all parameters

def hypo(x,sT): #hypothesis function
    return np.dot(x,sT)

def costFunction(x,sT): #cost function
    h = hypo(x,sT)
    return (1/(2*m))*sum(np.array((h-target))**2)
  
def gradientCost(x,sT): #gradient descent
    h = hypo(x,sT)
    return (1/(m))*np.dot(df.T,(h-target))

กำหนดให้ learning rate = 0.01 <br>
และ เริ่มทำการ train

In [0]:
alpha = 0.01 # learning rate
last = costFunction(df,sT)+1
while last - costFunction(df,sT)>0.001: #loop until change < 0.001
  last = costFunction(df,sT)
  sT = sT - alpha*(gradientCost(df,sT))
print('result :',sT.T[0])

result : [22.65365393 -0.06590317  0.64831977 -0.18460845  0.76100326 -1.12507124
  3.03972749 -0.11594876 -2.25233247  0.81150778 -0.56628214 -1.82116539
  0.88059889 -3.70432229]


สรุป เปรียบเทียบผลการทำนาย

In [0]:
compare = pd.DataFrame({'Target Prices':np.array(target).T[0],'Predicted Prices':np.dot(df,sT).T[0]}) # comparing result with target 
compare.head()

,Target Prices,Predicted Prices
0,24.0,30.595625
1,21.6,24.982998
2,34.7,30.979016
3,33.4,29.284144
4,36.2,28.673260
